# Create Aurora Borealis Art using GANs and PyTorch

Based on https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/README.rst and https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

In [1]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

arn:aws:iam::019326146125:role/service-role/AmazonSageMaker-ExecutionRole-20190407T165524


In [2]:
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch(entry_point='../train-sagemaker.py',
                            role=role,
                            train_instance_type='ml.p2.xlarge',
                            train_instance_count=1,
                            framework_version='1.0.0',
                            hyperparameters = {'epochs': 50,
                                              'output_path': 's3://markstrefford-art-1/aurora-model-1'})

* Now set up training a model

In [9]:
pytorch_estimator.fit({'training': 's3://markstrefford-art-1/aurora-art-1'})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-04-10-17-45-17-716


2019-04-10 17:45:18 Starting - Starting the training job...
2019-04-10 17:45:19 Starting - Launching requested ML instances......
2019-04-10 17:46:20 Starting - Preparing the instances for training......
2019-04-10 17:47:37 Downloading - Downloading input data
2019-04-10 17:47:37 Training - Downloading the training image......
2019-04-10 17:48:42 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-04-10 17:48:44,332 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-04-10 17:48:44,359 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-04-10 17:48:44,965 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-04-10 17:48:45,263 sagemaker-containers INFO     Module train-sagemaker does not provide a setup.py. 
Generating setup.py
2019-04-10 17:48:45,2


2019-04-10 17:49:08 Uploading - Uploading generated training model
2019-04-10 17:49:08 Completed - Training job completed
Billable seconds: 99


# From examples... 

https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/pytorch_lstm_word_language_model/pytorch_rnn.ipynb

In [3]:
%%bash
wget http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
unzip -n wikitext-2-raw-v1.zip
cd wikitext-2-raw
mv wiki.test.raw test && mv wiki.train.raw train && mv wiki.valid.raw valid

Archive:  wikitext-2-raw-v1.zip
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  


--2019-04-10 15:38:15--  http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
Resolving research.metamind.io.s3.amazonaws.com (research.metamind.io.s3.amazonaws.com)... 52.216.17.48
Connecting to research.metamind.io.s3.amazonaws.com (research.metamind.io.s3.amazonaws.com)|52.216.17.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip.1’

     0K .......... .......... .......... .......... ..........  1%  354K 13s
    50K .......... .......... .......... .......... ..........  2% 1.38M 8s
   100K .......... .......... .......... .......... ..........  3% 1.37M 6s
   150K .......... .......... .......... .......... ..........  4% 60.2M 5s
   200K .......... .......... .......... .......... ..........  5%  713K 5s
   250K .......... .......... .......... .......... ..........  6% 90.4M 4s
   300K .......... .......... .......... .......... ..........  7%  191M 3s
   350K .....

In [4]:
!head -5 wikitext-2-raw/train

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series 

In [5]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-rnn-lstm'

role = sagemaker.get_execution_role()

inputs = sagemaker_session.upload_data(path='wikitext-2-raw', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-eu-west-1-019326146125/sagemaker/DEMO-pytorch-rnn-lstm


In [6]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.0.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    source_dir='../source',
                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 6,
                        'tied': True
                    })

In [ ]:
estimator.fit({'training': inputs})

INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-04-10-15-38-24-889


2019-04-10 15:38:25 Starting - Starting the training job...
2019-04-10 15:38:26 Starting - Launching requested ML instances.........
2019-04-10 15:40:00 Starting - Preparing the instances for training....
2019-04-10 15:41:21 Downloading - Downloading input data
2019-04-10 15:41:21 Training - Downloading the training image......
2019-04-10 15:42:27 Training - Training image download completed. Training in progress..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-04-10 15:42:29,045 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-04-10 15:42:29,071 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-04-10 15:42:30,477 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-04-10 15:42:30,775 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-04-10 15:42:30,775 sagem

| epoch   5 |  1200/ 2983 batches | lr 20.00 | ms/batch 92.66 | loss  4.98 | ppl   145.48
| epoch   5 |  1400/ 2983 batches | lr 20.00 | ms/batch 93.19 | loss  5.06 | ppl   156.83
| epoch   5 |  1600/ 2983 batches | lr 20.00 | ms/batch 92.05 | loss  5.13 | ppl   168.43
